In [25]:
import requests
import json
import chardet
import re
import pandas as pd
import numpy as np
from pprint import pprint
import requests

import dm_pb2 as Danmaku


In [26]:
# 1.根据bvid请求得到cid

def get_cid():
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=BV1ct4y1n7t9'
    res = requests.get(url).text
    json_dict = json.loads(res)
    pprint(json_dict)
    return json_dict["data"][0]["cid"]
cid = get_cid()
print(cid)



{'code': 0,
 'data': [{'cid': 807427488,
           'dimension': {'height': 2160, 'rotate': 0, 'width': 3840},
           'duration': 60,
           'first_frame': 'http://i1.hdslb.com/bfs/storyff/n220818qn1r63g1judtydg1wvft368d2_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '【4K顶级画质60FPS】蔡徐坤《只因你太美》原版完整版现场！一晃眼6年过去了',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
807427488


In [33]:
# 更新：新版api需要使用proto获取弹幕文件
def get_prot_dm(cid):
    url = 'https://api.bilibili.com/x/v2/dm/web/seg.so'
    params = {
        'type': 1,         # 弹幕类型
        'oid': cid,    # cid
        'segment_index': 1 # 弹幕分段
    }
    resp = requests.get(url, params)
    data = resp.content

    danmaku_seg = Danmaku.DmSegMobileReply()
    danmaku_seg.ParseFromString(data)

    return danmaku_seg.elems

danmu_proto = get_prot_dm(cid)
print(len(danmu_proto))

2583


In [39]:
#得到弹幕 array
def to_list(prot):
    l = []
    for i in range(len(prot)):
        l.append({})
        l[i]["id"] = prot[i].id
        l[i]["progress"] = prot[i].progress
        l[i]["content"] = prot[i].content

    return l
listed_danmu = to_list(danmu_proto)
df = pd.DataFrame(listed_danmu)
print(df)
df.to_csv("video.csv")

                       id  progress                               content
0     1122558100960985600      2165                           这个鼻子反光，笑死我了
1     1123701354657768192       519       b站冷知识：点赞和关注是免费的，却可以让做视频的up开心好几天
2     1125201605340467968     21367                                  口技细节
3     1126748923101412352     13483                                  太清晰了
4     1127315387663627520     27516                                  白带异常
...                   ...       ...                                   ...
2578  1308734639422320384     26929                         重回巅峰！！！！！！！！！
2579  1308822999214058752     12045  ——————————————《梦开始的地方》——————————————
2580  1308846297138145280     52004                              《梦开始的地方》
2581  1309195021874303744     58901                               不愧是我家哥哥
2582  1309257063397989120      9685                      徐坤居然还戴着耳坠，以前没注意到

[2583 rows x 3 columns]


In [28]:
# 2.根据cid请求弹幕，解析弹幕得到最终的数据

# def get_data(cid):
#     final_url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
#     final_res = requests.get(final_url)
#     final_res.encoding = chardet.detect(final_res.content)['encoding']
#     final_res = final_res.text
#     pattern = re.compile('<d.*?>(.*?)</d>')
#     match = re.compile(r'<d\s+[^>]*\bp="([^"]*)"')
#     pa = match.findall(final_res)
#
#     data = pattern.findall(final_res)
#     danmu_time = [float(item.split(',')[0]) for item in pa]
#     danmu_mode = [float(item.split(',')[1]) for item in pa]
#     danmu_size = [float(item.split(',')[2]) for item in pa]
#     danmu_color = [float(item.split(',')[3]) for item in pa]
#     danmu_abstime = [float(item.split(',')[4]) for item in pa]
#     danmu_pool = [float(item.split(',')[5]) for item in pa]
#     danmu_id = [(item.split(',')[6]) for item in pa]
#     danmu_rowid = [float(item.split(',')[7]) for item in pa]
#     print(len(data))
#     return (data, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid)
#
#
# (danmu_text, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid) = get_data(cid)

In [29]:
# # 分割弹幕为八个部分
# # 视频持续时间
# max_time = max(danmu_time)
# divided_danmu = []
# for i in range(8):
#     time_start = max_time * (i/8)
#     time_stop = max_time * ((i+1)/8)
#     for j in range(len(danmu_time)):
#         if time_start <= danmu_time[j] <= time_stop:
#             divided_danmu.append([i, danmu_text[j]])
# print(len(danmu_time))

In [30]:
# export csv
# arr = np.array(divided_danmu)
# df = pd.DataFrame(arr)
# df.to_csv("video.csv", header=["label", "text"])

In [31]:
import requests

# 设置请求参数
params = {
    "bvid": "BV1J4411v7g6",  # 你想获取推荐视频的原始视频的aid
    "num": 1,  # 获取的推荐视频数量，最多为40
}

# 发送请求
response = requests.get("https://api.bilibili.com/x/web-interface/archive/related", params=params)

# 解析响应
if response.status_code == 200:
    data = response.json()
    recommend_list = data.get("data")
    # 处理推荐视频列表数据
    print(len(recommend_list))
else:
    print("请求失败")


40


In [32]:
#迭代获取其他视频的推荐
for i

SyntaxError: invalid syntax (3688025231.py, line 2)

In [ ]:
# 3.保存弹幕列表

def save_to_file(data):
    with open("dan_mu.txt", mode="w", encoding="utf-8") as f:
        for i in data:
            f.write(i)
            f.write("\n")
save_to_file(data)


In [ ]:
# 1 导入相关库
import pandas as pd
import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from imageio import imread

import warnings
warnings.filterwarnings("ignore")

# 2 读取文本文件，并使用lcut()方法进行分词
with open("dan_mu.txt",encoding="utf-8") as f:
    txt = f.read()
txt = txt.split()
data_cut = [jieba.lcut(x) for x in txt]
data_cut
# 3 读取停用词
stop = [" ","道","说道","说",'了']
# 4 去掉停用词之后的最终词
s_data_cut = pd.Series(data_cut)
all_words_after = s_data_cut.apply(lambda x:[i for i in x if i not in stop])
# 5 词频统计
all_words = []
for i in all_words_after:
    all_words.extend(i)
word_count = pd.Series(all_words).value_counts()
# 6 词云图的绘制
# 1）读取背景图片


# 2）设置词云参数
wc = WordCloud(font_path="C:/Users/Windows/fonts/simhei.ttf",
               background_color="white",
               max_words=2000,
               max_font_size=200,
               random_state=42
              )
wc2 = wc.fit_words(word_count)

# 3）绘制词云图
plt.figure(figsize=(16,8), dpi=300)
plt.imshow(wc2)
plt.axis("off")
plt.show()
wc.to_file("ciyun.jpg")

